Benefit of being able to help people sift through myriad availibilities. 

# DATA UNDERSTANDING

What does the data look like?

What information can we glean?

In [1]:
# from pyspark.sql import SparkSession
from pyspark.sql import Window
from pyspark.sql.functions import percent_rank
from pyspark.ml.recommendation import ALS
# spark = SparkSession.builder.master("local[*]").getOrCreate()

In [2]:
import pyspark
from pyspark.sql import SQLContext
sc = pyspark.SparkContext('local[*]')
sql_context = SQLContext(sc)

Up the memory to try to prevent java heap space OutOfMemory errors

In [3]:
sc._conf.set('spark.executor.memory', '16g')
sc._conf.getAll()

[('spark.executor.memory', '16g'),
 ('spark.app.id', 'local-1567623362536'),
 ('spark.driver.host', '10.237.132.213'),
 ('spark.rdd.compress', 'True'),
 ('spark.driver.port', '58665'),
 ('spark.serializer.objectStreamReset', '100'),
 ('spark.master', 'local[*]'),
 ('spark.executor.id', 'driver'),
 ('spark.submit.deployMode', 'client'),
 ('spark.ui.showConsoleProgress', 'true'),
 ('spark.app.name', 'pyspark-shell')]

In [4]:
df = sql_context.read.format('com.databricks.spark.csv').options(header='true', inferschema='true').load('data/ratings.csv')

In [5]:
df.count()

26024289

In [6]:
df.columns

['userId', 'movieId', 'rating', 'timestamp']

In [7]:
df

DataFrame[userId: int, movieId: int, rating: double, timestamp: int]

In [8]:
df.take(3)

[Row(userId=1, movieId=110, rating=1.0, timestamp=1425941529),
 Row(userId=1, movieId=147, rating=4.5, timestamp=1425942435),
 Row(userId=1, movieId=858, rating=5.0, timestamp=1425941523)]

#### Train Test Split by date at 10 most recent percent of the data

In [9]:
time_ranked_df = df.withColumn("rank", percent_rank().over(Window.partitionBy().orderBy("timestamp")))

In [10]:
train_df = time_ranked_df.where('rank <= 0.9').drop('rank')
test_df = time_ranked_df.where('rank > 0.9').drop('rank')

In [11]:
train_df.persist()
test_df.persist()

DataFrame[userId: int, movieId: int, rating: double, timestamp: int]

In [12]:
f"""
Train rows: {train_df.count()}
Test rows: {test_df.count()}
"""

'\nTrain rows: 23421860\nTest rows: 2602429\n'

#### Sanity Test: On this processor how feasable is ALS model on 23 mil rows

In [13]:
als = ALS(userCol='userId' ,
          itemCol='movieId' ,
          ratingCol='rating'
         )

In [14]:
sc.setCheckpointDir('/tmp')

In [15]:
model = als.fit(train_df)

Py4JJavaError: An error occurred while calling o72.fit.
: org.apache.spark.SparkException: Job aborted due to stage failure: Task 0 in stage 14.0 failed 1 times, most recent failure: Lost task 0.0 in stage 14.0 (TID 56, localhost, executor driver): java.lang.OutOfMemoryError: Java heap space
	at scala.collection.mutable.ArrayBuilder$ofInt.mkArray(ArrayBuilder.scala:335)
	at scala.collection.mutable.ArrayBuilder$ofInt.resize(ArrayBuilder.scala:341)
	at scala.collection.mutable.ArrayBuilder$ofInt.ensureSize(ArrayBuilder.scala:353)
	at scala.collection.mutable.ArrayBuilder$ofInt.$plus$eq(ArrayBuilder.scala:358)
	at scala.collection.mutable.ArrayBuilder$ofInt.$plus$eq(ArrayBuilder.scala:328)
	at org.apache.spark.ml.recommendation.ALS$UncompressedInBlockBuilder.add(ALS.scala:1383)
	at org.apache.spark.ml.recommendation.ALS$$anonfun$23$$anonfun$apply$16.apply(ALS.scala:1597)
	at org.apache.spark.ml.recommendation.ALS$$anonfun$23$$anonfun$apply$16.apply(ALS.scala:1596)
	at scala.collection.It

# ~ ~ Heap Space Memory Error ~ ~

# DATA PREPARATION

Train Test Split

# MODELING

PySpark Alternating Least Squares

# EVALUATION

Would evaluate in ... manner.

# DEPLOYMENT

Would deploy in ... manner.